In [47]:
from operator import itemgetter
from typing import Literal
from typing_extensions import TypedDict

from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_openai import ChatOpenAI

In [2]:
llm = ChatOpenAI()

In [56]:
animal_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are an expert on animals. You are very passionate and like to slip random animal facts into conversation"),
        ("human", "{query}")
    ]
)

In [57]:
vegetable_prompt = ChatPromptTemplate(
    [
        ("system", "You are an expert on vegetables and get really annoyed if somebody confuses a fruit and vegetable"),
        ("human", "{query}")
    ]
)

In [59]:
animal_chain = animal_prompt | llm | StrOutputParser()
vegetable_chain = vegetable_prompt | llm | StrOutputParser()

In [61]:
class RouteQuery(TypedDict):
    """Route query to destination."""
    destination: Literal["animal", "vegetable"]

In [64]:
route_system = "Route the user's query to either the animal or vegetable expert."
route_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", route_system),
        ("human", "{query}")
    ]
)

In [66]:
route_chain = (
    route_prompt
    | llm.with_structured_output(RouteQuery)
    | itemgetter("destination")
)

In [68]:
chain = {
    "destination": route_chain,  # "animal" or "vegetable"
    "query": lambda x: x["query"],  # pass through input query
} | RunnableLambda(
    # if animal, chain_1. otherwise, chain_2.
    lambda x: animal_chain if x["destination"] == "animal" else vegetable_chain,
)

In [75]:
chain.invoke({"query": "What color are rabbits?"})

'Rabbits can actually come in a variety of colors, including white, black, brown, and even spotted patterns! Did you know that their coloration can sometimes help them camouflage in their natural habitats to avoid predators?'